![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)

<p><font size="5" color='grey'> <b>
Grundlagen Modellansteuerung
</b></font> </br></p>

---

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  Colab-Umfeld</font> </br></p>
# Installierte Python Version
import sys
print(f"Python Version: ",sys.version)
# Installierte LangChain Bibliotheken
print()
print("Installierte LangChain Bibliotheken:")

!pip list | grep '^langchain'
# Unterdrückt die "DeprecationWarning" von LangChain für die Memory-Funktionden
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="langsmith.client")

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  SetUp API-Keys (setup_api_keys)</font> </br></p>
def setup_api_keys():
    """Konfiguriert alle benötigten API-Keys aus Google Colab userdata"""
    from google.colab import userdata
    import os
    from os import environ

    # Dictionary der benötigten API-Keys
    keys = {
        'OPENAI_API_KEY': 'OPENAI_API_KEY',
        'HF_TOKEN': 'HF_TOKEN',
        # Weitere Keys bei Bedarf
    }

    # Keys in Umgebungsvariablen setzen
    for env_var, key_name in keys.items():
        environ[env_var] = userdata.get(key_name)

    return {k: environ[k] for k in keys.keys()}

# Verwendung
all_keys = setup_api_keys()
# Bei Bedarf einzelne Keys direkt zugreifen
# WEATHER_API_KEY = all_keys['WEATHER_API_KEY']

# 1 | Überblick
---


Die rasante Entwicklung im Bereich der künstlichen Intelligenz (KI) hat zu beeindruckenden Fortschritten bei großen Sprachmodellen (LLMs) geführt. Diese Modelle können menschenähnlichen Text generieren, Sprachen übersetzen und komplexe Fragen beantworten. Doch wie genau steuert man diese Modelle, um die gewünschten Ergebnisse zu erzielen? In diesem Modul werden drei gängige Methoden der Modellansteuerung untersucht: Prompting, Retrieval-Augmented Generation (RAG) und Fine-Tuning.



# 2 | Prompting
---

Prompting ist die einfachste und direkteste Methode zur Steuerung von LLMs. Dabei wird dem Modell eine Textaufforderung, der sogenannte `Prompt`, gegeben, der die gewünschte Ausgabe beschreibt. Die Qualität der Ausgabe hängt stark von der Qualität des Prompts ab. Ein gut formulierter Prompt sollte klar, prägnant und spezifisch sein.


[Say What You See](https://artsandculture.google.com/experiment/say-what-you-see/jwG3m7wQShZngw)

<p><font color='black' size="5">
Prompt-Elemente
</font></p>

Eingabeaufforderungen bestehen in der Regel aus vier wesentlichen Elementen, die je nach Aufgabe variieren können. Die Hauptbestandteile sind:

* **Anweisungen** – Definieren die Aufgabe des Modells, entweder als Aufgabenbeschreibung oder als konkrete Handlungsanweisung.  
* **Kontext** – Liefert ergänzende Informationen, um das Modell bei der Bearbeitung zu unterstützen.  
* **Eingabedaten** – Enthalten die spezifischen Informationen, zu denen eine Antwort generiert werden soll.  
* **Ausgabeindikator** – Signalisiert den Übergang zur erwarteten Modellantwort.  

Weitere Beispiele zur Anwendung dieser Struktur folgen später.

Nachfolgend ein Beispiel für eine Eingabeaufforderung zur Erstellung einer Produktzusammenfassung.

```
Fassen Sie die folgende Produktbewertung in einem Satz zusammen:
Produkt: Intelligente digitale Personenwaage für Körpergewicht, Fett, BMI, Muskelmassezusammensetzung
Produktbewertung: Diese Waage ist fantastisch! Der Einrichtungsvorgang war schnell und einfach und
dauerte nur wenige Minuten. Ich konnte die App kostenlos auf mein etwas veraltetes iPhone 8
herunterladen. Die Funktion, bei der Ihr Gewicht sofort mit der App synchronisiert wird, macht die
Nachverfolgung mühelos. Die Funktionen zur Essensplanung und Kalorienzählung der App sind unglaublich
benutzerfreundlich. Ich bin absolut begeistert! Außerdem hat die Waage ein elegantes, modernes
Erscheinungsbild, das wirklich attraktiv ist.


Zusammenfassung: ...
```

Diese Eingabeaufforderung erzeugt möglicherweise die folgende Ausgabe:

```
Die intelligente digitale Personenwaage wird für ihre schnelle Einrichtung, die einfache
Synchronisierung mit ihrer App auf älteren Telefonen zur mühelosen Nachverfolgung, die
benutzerfreundlichen Funktionen zur Essensplanung und Kalorienzählung sowie ihr schlankes, modernes
Design hoch gelobt.
```

<p><font color='black' size="5">
Code-Beispiel
</font></p>

In [ ]:
from openai import OpenAI, chat
from IPython.display import display, Markdown

client = OpenAI()

user_prompt = """
Fassen Sie die folgende Produktbewertung in einem Satz zusammen:
Produkt: Intelligente digitale Personenwaage für Körpergewicht, Fett, BMI, Muskelmassezusammensetzung
Produktbewertung: Diese Waage ist fantastisch! Der Einrichtungsvorgang war schnell und einfach und
dauerte nur wenige Minuten. Ich konnte die App kostenlos auf mein etwas veraltetes iPhone 8
herunterladen. Die Funktion, bei der Ihr Gewicht sofort mit der App synchronisiert wird, macht die
Nachverfolgung mühelos. Die Funktionen zur Essensplanung und Kalorienzählung der App sind unglaublich
benutzerfreundlich. Ich bin absolut begeistert! Außerdem hat die Waage ein elegantes, modernes
Erscheinungsbild, das wirklich attraktiv ist.
"""

completion = chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "Du bist ein hilfreicher KI-Assistent."},
    {"role": "user", "content": user_prompt}
  ]
)

display(Markdown('## 🤖 KI:'))
display(Markdown(completion.choices[0].message.content))

# 3 | Retrieval-Augmented Generation
---

RAG (Retrieval-Augmented Generation) ist eine **hybride Methode**, die die Stärken von **LLMs** mit denen von **Information-Retrieval-Systemen** **kombiniert**. Dabei greift das LLM auf llm-externe Wissensdatenbanken zu, um seine Antworten zu verbessern und die Probleme von Halluzinationen und veralteten Informationen zu mindern.

**Funktionsweise:**

+ Der Benutzer gibt eine Anfrage in das RAG-System ein.
+ Das System sucht in externe Wissens nach relevanten Informationen.
+ Die relevanten Informationen werden dem LLM als Kontext bereitgestellt.
+ Das LLM generiert eine Antwort basierend auf der Anfrage **und** dem Kontext.


**Einsatzszenarien:**

RAG eignet sich besonders für Aufgaben, die aktuelle oder domänenspezifische Informationen erfordern, wie z. B.:

+ Kundensupport mit Zugriff auf Produktdatenbanken
+ Medizinische Diagnose mit Zugriff auf aktuelle Forschungsergebnisse
+ Finanzplanung mit Zugriff auf Marktdaten
+ Beantwortung von Fragen zu Unternehmensinformationen



# 4 | Fine-Tuning
---

Fine-Tuning ist eine Methode, bei der ein bereits **trainiertes LLM** auf einer kleineren, **aufgabenspezifischen** Datenmenge weiter trainiert wird. Dadurch kann das Modell an **spezifische** **Anforderungen** angepasst und seine Leistung für diese Aufgaben verbessert werden.

**Funktionsweise:**

+ Ein vortrainiertes LLM wird ausgewählt.
+ Eine aufgabenspezifische Datenmenge wird vorbereitet.
+ Das LLM wird auf dieser Datenmenge trainiert.
+ Die Parameter des LLM werden angepasst, um die Leistung für die spezifische Aufgabe zu optimieren.


**Einsatzszenarien:**

Fine-Tuning eignet sich besonders für Aufgaben, die eine hohe Genauigkeit und **Domänenspezialisierung** erfordern, wie z. B.:

+ fachlich spezialisierte LLMs (Recht, Medizin, ...)
+ Sentimentanalyse
+ Textklassifizierung
+ Spam-Erkennung
+ Personalisierte Kundeninteraktionen




# 5 | Entscheidungskriterien
---

Die Wahl der geeigneten Methode hängt von verschiedenen Faktoren ab, darunter:


| Kriterium | Prompting | RAG | Fine-Tuning |
|-----------|-----------|-----|-------------|
| **Komplexität** | Niedrig<br><br>Die Implementierung ist <br>einfach und erfordert keine <br>Änderungen am Modell. | Mittel<br><br>RAG erfordert die Integration<br>von Retrieval-Mechanismen<br>und einer Wissensdatenbank. | Hoch<br><br>Fine-Tuning beinhaltet<br>einen komplexeren Prozess<br>des weiteren Trainings. |
| **Effizienz** | Hoch<br><br>Prompting ermöglicht eine<br>schnelle und flexible<br>Interaktion mit dem Modell. | Mittel<br><br>RAG benötigt zusätzliche<br>Schritte für den Datenabruf,<br>was die Effizienz mindert. | Niedrig<br><br>Fine-Tuning ist rechen-<br>intensiv und zeitaufwendig. |
| **Genauigkeit** | Niedrig<br><br>Die Genauigkeit hängt stark<br>von der Qualität des<br>Prompts ab. | Mittel<br><br>RAG bietet höhere<br>Genauigkeit durch externe<br>Informationen. | Hoch<br><br>Führt zu höherer<br>Genauigkeit bei<br>spezifischen Aufgaben. |
| **Flexibilität** | Hoch<br><br>Sehr flexibel und für<br>verschiedene Aufgaben<br>einsetzbar. | Mittel<br><br>Flexibler als Fine-Tuning,<br>aber weniger flexibel als<br>Prompting. | Niedrig<br><br>Ist auf bestimmte Aufgaben<br>spezialisiert und weniger<br>anpassungsfähig. |
| **Ressourcen-<br>bedarf** | Niedrig<br><br>Benötigt nur minimale<br>Ressourcen für die<br>Ausführung. | Mittel<br><br>Ressourcen für Datenbank-<br>verwaltung und Retrieval<br>erforderlich. | Hoch<br><br>Erfordert erhebliche<br>Rechenleistung und Zeit<br>für Training. |
| **Datenbedarf** | Niedrig<br><br>Keine zusätzlichen<br>Trainingsdaten<br>erforderlich. | Mittel<br><br>Benötigt Wissensdatenbank<br>mit relevanten<br>Informationen. | Hoch<br><br>Benötigt große Mengen an<br>spezifischen<br>Trainingsdaten. |
| **Beschreibung** | Einfache Methode mit<br>Textanweisungen für das<br>Modell. | Kombination von LLMs mit<br>Information-Retrieval-<br>Systemen. | Weiteres Training des<br>Modells auf spezifischen<br>Datensätzen. |



# 6 | Auswahl Modellansteuerung

![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/GenAI/rag_klein.jpg)





Bei der Wahl zwischen den verschiedenen Modellansteuerungstechniken gibt es immer Trade-offs, die berücksichtigt werden müssen:

- **Kosteneffizienz**: Fine-Tuning ist kostspieliger, da es nicht nur erhebliche Rechenressourcen erfordert, sondern auch Zeit, um ein Modell zu trainieren. RAG bietet einen Mittelweg, da es externe Daten nutzt, ohne dass ein vollständiges Modell neu trainiert werden muss. Prompting ist hingegen am kostengünstigsten, da es keine zusätzlichen Trainingsphasen erfordert.

- **Flexibilität**: Fine-Tuning bietet die höchste Flexibilität, da es das Modell vollständig an die speziellen Anforderungen anpassen kann. RAG bietet ebenfalls Flexibilität, allerdings hängt die Qualität der Ergebnisse stark von der Qualität der externen Daten ab. Prompting ist flexibler in der Formulierung der Anfragen, hat jedoch in komplexen Szenarien oft Einschränkungen in der Präzision und Tiefe der Antworten.

- **Generalität vs. Spezifität**: Modelle, die nur mit Prompting betrieben werden, sind in der Regel allgemeiner und weniger präzise in der Bearbeitung spezifischer Anfragen. Fine-Tuning ermöglicht die Anpassung an spezifische Anforderungen, reduziert jedoch die Generalisierbarkeit auf andere Domänen. RAG bietet eine Balance, da es externe Daten zur Verbesserung der Antwortqualität verwendet, ohne die Generalität des zugrundeliegenden Modells zu stark zu beeinflussen.

